# **Batch Process Segmentations**

***Prior to this notebook, you should have already run through [1.0_image_setup](1.0_image_setup.ipynb) and determined the best settings for your organelles of interest (minimum 1) and the masks.***

In this notebook, the settings determined in notebooks 1.1 through 1.7 we be used to batch process the segmentations for multiple images. The steps included here correlate to the `Batch Processor` module of the `infer-subc` Napari plugin called `organelle-segmenter-plugin`.

### ⚙️ **Suggested Setup**
We suggest that each the images from one experimental replicate be processed with the same settings, if possible. For analysis in `Part 2 - Organelle Quantification`, the segmentation files for each organelle across all images in an experimental replicate should be saved in the same folder.

### ➡️ **Input**
The following files and information will be necessary for batch processing segmentations:
1. "Raw" confocal microscopy image (".tiff", ".tif", or ".czi") where each channel of the image represents one of the organelles being segmented
2. The segementation settings for each workflow you wish to include in the batch processing (*Hint: these are deteremined in notebooks 1.1 through 1.7*)

All of the organelles and the masks can be processed in the same batch processing session. However, if you wish to only include a subset that is fine. Keep in mind that the minimum necessary input for quantification is one masks workflow output and one organelle workflow output.

### **Output** ➡️
The batch processing function will output one ".tiff" segmentation file per workflow per image. They will be saved into the same output folder. The naming convention is to append the oragnelle or masks "nickname" used in the Napari plugin to the end of the raw file name.

### 🔎 **Quality Check and Mask Separation**
After segmenting all the cells in your dataset, we recommend you quality check your segmentation results by visually inspecting the images. The Segmentation Validation pipeline is included in the [Full Quantification Pipeline Notebook](/notebooks/part_2_quantification/full_quantification_pipeline.ipynb) to streamline the validation process.

-----

### 👣 **Summary of steps**  

➡️ **BATCH PROCESSING**
- **`Step 1`** - Define the paths to your data

- **`Step 2`** - Define the settings for each workflow being used

    Workflow options include:
    - Infer [`masks`](./1.1_infer_masks_from-composite_with_nuc.ipynb)
    - Infer [`masks A`](./1.1a_infer_masks_from-composite_single_cell.ipynb)
    - Infer [`masks B`](./1.1b_infer_masks_from-composite_multiple-cells.ipynb)
    - Infer [`masks C`](./1.1c_infer_masks_from-composite_neuron_with_pm.ipynb)
    - Infer [`masks D`](./1.1d_infer_masks_from-composite_ipsc.ipynb)
    - Infer [`lysosomes`](1.2_infer_lysosome.ipynb)
    - Infer [`mitochondria`](1.3_infer_mitochondria.ipynb)
    - Infer [`golgi`](1.4_infer_golgi.ipynb)
    - Infer [`peroxisomes`](1.5_infer_peroxisome.ipynb)
    - Infer [`endoplasmic reticulum (ER)`](1.6_infer_ER.ipynb)
    - Infer [`lipid droplets`](1.7_infer_lipid_droplet.ipynb)

- **`Step 3`** - Run batch processing

---------------------
## **IMPORTS**
Details about the functions included in this subsection are outlined in the [`1.0_image_setup`](1.0_image_setup.ipynb) notebook. Please visit that notebook first if you are confused about any of the code included here.

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
from pathlib import Path
from typing import List, Union
import os
import time

from infer_subc.core.file_io import (read_czi_image,
                                     export_inferred_organelle,
                                     list_image_files)
from infer_subc.organelles import *
from infer_subc.utils.batch import batch_process_segmentation

-----

## **BATCH PROCESSING**

### **`STEP 1` - Define the paths to your data**

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following information about your data: 
- `raw_path`: A string or a Path object of the path to your raw (e.g., intensity) images that will be the input for segmentation
- `raw_file_type`: The raw file type (e.g., ".tiff" or ".czi")
- `seg_path`: A string or a Path object of the path where the segmentation outputs will be saved
- `file_splitter`: An optional string to include before the segmentation suffix at the end of the output file. For example, if the file_splitter was "20240105", the segmentation file output from the 1.1_masks workflow would include: "{base-file-name}-20240105-masks".

In [ ]:
#### USER INPUT REQUIRED ###
raw_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc/raw_single"
raw_file_type = ".czi"
seg_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc/out_test_single"
file_splitter = None

### **`STEP 2` - Define the settings for each workflow being used**

#### &#x1F6D1; &#x270D; **User Input Required:**

For each workflow that you wish to include in the batch processing, please fill out the information in the associated settings list. 
The necessary settings for each function are included below.



For infer_masks:
```mermaid
- `masks_settings` = [nuc_ch: Union[int,None], nuc_median_sz: int, nuc_gauss_sig: float,
                nuc_thresh_factor: float, nuc_thresh_min: float, nuc_thresh_max: float,
                nuc_min_hole_w: int, nuc_max_hole_w: int, nuc_small_obj_w: int,
                nuc_fill_filter_method: str, cell_weights: list[int], cell_rescale: bool,
                cell_median_sz: int, cell_gauss_sig: float, cell_mo_method: str,
                cell_mo_adjust: float, cell_mo_cutoff_size: int, cell_min_hole_w: int,
                cell_max_hole_w: int, cell_small_obj_w: int, cell_fill_filter_method: str,
                cell_watershed_method: str, cyto_erode_nuclei = True]
```

For infer_masks_A:
```mermaid
- `masks_A_settings` = [cyto_weights: list[int], cyto_rescale: bool, cyto_median_sz: int,
                    cyto_gauss_sig: float, cyto_mo_method: str, cyto_mo_adjust: float,
                    cyto_mo_cutoff_size: int, cyto_min_hole_w: int, cyto_max_hole_w: int,
                    cyto_small_obj_w: int, cyto_fill_filter_method: str, nuc_min_hole_w: int,
                    nuc_max_hole_w: int, nuc_fill_method: str, nuc_small_obj_w: int,
                    nuc_fill_filter_method: str, cell_min_hole_width: int, cell_max_hole_width: int,
                    cell_small_obj_width: int, cell_fill_filter_method: str]
```

For infer_masks_B:
```mermaid
- `masks_B_settings` = [cyto_weights: list[int], cyto_rescale: bool, cyto_median_sz: int,
                    cyto_gauss_sig: float, cyto_mo_method: str, cyto_mo_adjust: float,
                    cyto_mo_cutoff_size: int, cyto_min_hole_w: int, cyto_max_hole_w: int,
                    cyto_small_obj_w: int, cyto_fill_filter_method: str, max_nuclei_width: int,
                    nuc_small_obj_width: int, cell_fillhole_max: int, cyto_small_object_width2: int]
```

For infer_masks_C:
```mermaid
- `masks_C_settings` = [pm_ch: Union[int,None], nuc_ch: Union[int,None], weights_I: list[int],
                   weights_II: list[int], invert_pm_I: bool, invert_pm_II: bool,
                   method_I: str, method_II: str, size_I: int,
                   size_II: int, global_method_I: str, global_method_II: str,
                   cutoff_size_I: int, cutoff_size_II: int, local_adj_I: float,
                   local_adj_II: float, bind_to_pm_I: bool, bind_to_pm_II: bool,
                   thresh_adj_I: float, thresh_adj_II: float, nuc_med_filter_size: int,
                   nuc_gaussian_smoothing_sigma: float, nuc_threshold_factor: float, nuc_thresh_min: float,
                   nuc_thresh_max: float, nuc_hole_min_width: int, nuc_hole_max_width: int,
                   nuc_small_object_width: int, nuc_fill_filter_method: str, nuc_search_img: str,
                   cm_method_I: str, cm_method_II: str, cm_size_I: int,
                   cm_size_II: int, cm_min_hole_width_I: int, cm_min_hole_width_II: int,
                   cm_max_hole_width_I: int, cm_max_hole_width_II: int, cm_small_obj_width_I: int,
                   cm_small_obj_width_II: int, cell_watershed_method: str, cell_min_hole_width: int,
                   cell_max_hole_width: int, cell_method: str, cell_size: int]
```

For infer_masks_D:
```mermaid
- `masks_D_settings` = [pm_ch: Union[int,None], nuc_ch: Union[int,None], weights: list[int],
                   median_sz: int, gauss_sig: float, thresh_factor: float,
                   thresh_min: float, thresh_max: float, min_hole_w: int,
                   max_hole_w: int, small_obj_w: int, fill_filter_method: str,
                   invert_pm: bool, cell_method: str, hole_min: int,
                   hole_max: int, fill_2d: bool]
```

For infer_lyso:
```mermaid
- `lyso_settings` = [lyso_ch: int, median_sz: int, gauss_sig: float,
                dot_scale_1: float, dot_cut_1: float, dot_scale_2: float,
                dot_cut_2: float, dot_scale_3: float, dot_cut_3: float,
                dot_method: str, fil_scale_1: float, fil_cut_1: float,
                fil_scale_2: float, fil_cut_2: float, fil_scale_3: float, 
                fil_cut_3: float, fil_method: str, min_hole_w: int,
                max_hole_w: int, small_obj_w: int, fill_filter_method: str]
```

For infer_mito:
```mermaid
- `mito_settings` = [mito_ch: int, median_sz: int, gauss_sig: float,
                    dot_scale_1: float, dot_cut_1: float, dot_scale_2: float,
                    dot_cut_2: float, dot_scale_3: float, dot_cut_3: float,
                    dot_method: str, fil_scale_1: float, fil_cut_1: float,
                    fil_scale_2: float, fil_cut_2: float, fil_scale_3: float, 
                    fil_cut_3: float, fil_method: str, min_hole_w: int,
                    max_hole_w: int, small_obj_w: int, fill_filter_method: str]
```

For infer_golgi:
```mermaid
- `golgi_settings` = [golgi_ch: int, median_sz: int, gauss_sig: float,
                    mo_method: str, mo_adjust: float, mo_cutoff_size: int,
                    min_thickness: int, thin_dist: int, dot_scale_1: float,
                    dot_cut_1: float, dot_scale_2: float, dot_cut_2: float,
                    dot_scale_3: float, dot_cut_3: float, dot_method: str,
                    min_hole_w: int, max_hole_w: int, small_obj_w: int,
                    fill_filter_method: str]
```

For infer_perox:
```mermaid
- `perox_settings` = [perox_ch: int, median_sz: int, gauss_sig: float,
                    dot_scale_1: float, dot_cut_1: float, dot_scale_2: float,
                    dot_cut_2: float, dot_scale_3: float, dot_cut_3: float,
                    dot_method: str, hole_min_width: int, hole_max_width: int,
                    small_object_width: int, fill_filter_method: str]
```

For infer_ER:
```mermaid
- `ER_settings` = [ER_ch: int, median_sz: int, gauss_sig: float,
            MO_thresh_method: str, MO_cutoff_size: float, MO_thresh_adj: float,
            fil_scale_1: float, fil_cut_1: float, fil_scale_2: float, 
            fil_cut_2: float, fil_scale_3: float, fil_cut_3: float,
            fil_method: str, min_hole_w: int, max_hole_w: int,
            small_obj_w: int, fill_filter_method: str]
```

For infer_LD:
```mermaid
- `LD_settings` = [LD_ch: str, median_sz: int, gauss_sig: float,
                method: str, thresh_factor: float, thresh_min: float,
                thresh_max: float, min_hole_w: int, max_hole_w: int,
                small_obj_w: int, fill_filter_method: str]
```

In [ ]:
#### USER INPUT REQUIRED ###
masks_settings = None
masks_A_settings = None
masks_B_settings = None
masks_C_settings = None
masks_D_settings = None
lyso_settings = None
mito_settings = None
golgi_settings = None
perox_settings = None
ER_settings = None
LD_settings = [6, 2, 1.34, "otsu", 0.8, 0.05, 1.0, 0, 2.5, 1, "3D"]

### **`STEP 3` - Run batch processing**

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block uses the inputs provided above to run the batch processing. The output of the cell will state when each image is completed.

In [ ]:
batch = batch_process_segmentation(raw_path,
                                    raw_file_type,
                                    seg_path,
                                    file_splitter,
                                    masks_settings,
                                    masks_A_settings,
                                    masks_B_settings,
                                    masks_C_settings,
                                    masks_D_settings,
                                    lyso_settings,
                                    mito_settings,
                                    golgi_settings,
                                    perox_settings,
                                    ER_settings,
                                    LD_settings)

-------------
### ✅ **BATCH PROCESSING SEGMENTATIONS COMPLETE!**

Continue on to quality check the segmentation outputs and separating the masks files into separate `cell` and `nuc` segmentation files [***REQUIRED for quantification***] in the [quality_check_segmentations](quality_check_segmentations.ipynb) notebook.